In [ ]:
import cv2
import numpy as np
from IPython import display as display
import ipywidgets as ipw
import PIL
import random
import time
from io import BytesIO

class Particle:
    #Constructor
    def __init__(self, x, y):
        self.radius = 0.0
        self.x = x
        self.y = y
        self.isGrowing = True

    def draw(self, img):
        color = (255, 255, 255)
        cv2.circle(img, (int(self.x), int(self.y)), int(self.radius), color, -1)

#Creates all the possible particles in the space
def generateParticles(x0, y0, maxX, maxY):

    global spaceBetweenParticles

    xValues = []
    yValues = []
    particles = []

    xi = x0
    while xi > spaceBetweenParticles:
        xValues.append(xi)
        xi -= spaceBetweenParticles
    xi = x0
    while xi < maxX:
        xValues.append(xi)
        xi += spaceBetweenParticles
    yi = y0
    while yi > spaceBetweenParticles:
        yValues.append(yi)
        yi -= spaceBetweenParticles
    yi = y0
    while yi < maxY:
        yValues.append(yi)
        yi += spaceBetweenParticles

    xValues.sort()
    yValues.sort()

    xSize = len(xValues)
    ySize = len(yValues)

    for x in range(0, xSize):
        currParticles = []
        for y in range(0, ySize):
            currParticles.append(Particle(xValues[x], yValues[y]))
        particles.append(currParticles)
            
    return particles

def chooseRandomParticle(particles):
    xRandomPosition = random.randrange(1, len(particles) - 1)
    yRandomPosition = random.randrange(1, len(particles[0]) - 1)
    particles[xRandomPosition][yRandomPosition].radius = 1

def nextIteration(particles):
    global maxRadius, activeParticles, decayValue, increaseConstant

    neighbors = [[1,0], [0,1], [-1,0], [0,-1]]
    xParticles = len(particles)
    yParticles = len(particles[0])
    activeParticles = 0

    for x in range(0, xParticles):
        for y in range(0, yParticles):
            if particles[x][y].isGrowing:
                for xOffset, yOffset in neighbors:
                    #We add to the current particle the "force" aplyied by the neighbours.
                    if(0 <= x + xOffset < xParticles) and (0 <= y  + yOffset < yParticles):
                        particles[x][y].radius += increaseConstant * particles[x + xOffset][y + yOffset].radius
                
            #We reach the maximum radius of the current particle we don't need to use it again
            if particles[x][y].isGrowing and particles[x][y].radius >= maxRadius:
                particles[x][y].isGrowing = False
            
            #If the current particle is not growing we set the particle to the initial state (talking about "force")
            if not particles[x][y].isGrowing:
                if particles[x][y].radius <= 1.0:
                    particles[x][y].radius = 0

            #We count the number of particles that are in use
            if particles[x][y].radius > 1.0:
                activeParticles += 1
            
            #Decay
            particles[x][y].radius *= decayValue

def drawParticles(particles, img):

    xParticles = len(particles)
    yParticles = len(particles[0])

    for x in range(0, xParticles):
        for y in range(0, yParticles):
            particles[x][y].draw(img)

#Or rows
height = 500
#Or cols
width = 600
#Max particle radius
maxRadius = 10
#Max particle radius
spaceBetweenParticles = maxRadius * 2 + 1
activeParticles = 1
decayValue = 0.9
increaseConstant = 0.15

x0 = width // 2
y0 = (height) // 2

maxIterations = 200

wIm = ipw.Image()
display.display(wIm)

#We create our grid
image = np.zeros((height + spaceBetweenParticles, width + spaceBetweenParticles, 3), dtype = "uint8")
#Creating all the particles
#We add an offset in height because without that we can't see the leyend (Iteration: n)
particles = generateParticles(x0, y0, width, height - (30 + maxRadius))

xParticles = len(particles)
yParticles = len(particles[0])

drawParticles(particles, image)

for iteration in range(0, 200):
    image[:] = (0, 0, 0)
    
    if activeParticles == 0:
        for x in range(0, xParticles):
            for y in range(0, yParticles):
                particles[x][y].isGrowing = True

        chooseRandomParticle(particles)

    nextIteration(particles)
    drawParticles(particles, image)
    #Show the current iteration in the image
    cv2.putText(image, "Iteration: " + str(iteration + 1), (spaceBetweenParticles, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
    #Show the current iteration in the image
    cv2.putText(image, "Particles: " + str(activeParticles), (250 + spaceBetweenParticles, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    pilIm = PIL.Image.fromarray(image, mode = "RGB")

    with BytesIO() as fOut:
        pilIm.save(fOut, format = "png")
        byPng = fOut.getvalue()
        
    # set the png bytes as the image value; 
    # this updates the image in the browser.
    wIm.value = byPng  
    time.sleep(0.15)


Image(value=b'')

In [ ]:
import cv2
import numpy as np
from IPython import display as display
import ipywidgets as ipw
import PIL
from io import BytesIO
import math
import random
import time 

class Particle:
    MaxV = np.sqrt(2)

    def __init__(self, x, y, speedX, speedY, radius, WallParticle=False):
        self.position = np.array([float(x), float(y)])
        self.speed = np.array([float(speedX), float(speedY)])
        self.WallParticle = WallParticle
        self.force = np.array([0.0, 0.0])
        self.radius = radius

    #By definition an unit vector it's magnitude must be 1 otherwise it isn't a unit vector
    def normalizeVector(self, x):
        norm = np.linalg.norm(x)
        if norm == 0.0:
            return x * np.inf
        return x / norm

    def calculateForce(self, r2):
        if self.WallParticle == True:
            return np.array([0.0, 0.0])
        result = self.position - r2.position
        r12magnitude = np.linalg.norm(result)
        if r12magnitude <= r2.radius + self.radius:
            #We get the unitary vector and then we split by the magnitude of R where R is a new vector from r1 to r2
            return self.normalizeVector(result) / (r12magnitude ** 2) * wallParticlesSize * 20
        return np.array([0.0, 0.0])

    def updatePosition(self):
        if self.WallParticle == True:
            return
        self.position += self.speed
        return

    def updateSpeed(self):
        if self.WallParticle == True:
            return
        self.speed += self.force
        vmag = np.linalg.norm(self.speed)

        if vmag > self.MaxV:
            self.speed = self.normalizeVector(self.speed) * self.MaxV
        return

    def draw(self, x0, y0, img):
        if self.WallParticle == True:
            color = (0, 255, 255)
        else:
            color = (255, 255, 255)
        cv2.circle(img, (int(x0 + self.position[0]), int(y0 - self.position[1])), self.radius, color, -1)
        return


particles = []
def lineOfWallParticles(x1, y1, x2, y2, N):
    global particles
    x = np.linspace(x1, x2, N)
    y = np.linspace(y1, y2, N)
    for i in range(N):
        particles.append(Particle(x[i], y[i], 0, 0, wallParticlesSize, WallParticle=True))


wIm = ipw.Image()
display.display(wIm)

maxX = 500
maxY = 500
x0 = int(maxX / 2)
y0 = int(maxY / 2)
nParticles = 70
particleSize = 7
wallParticlesSize = 15

img = np.zeros((500, 500, 3), dtype="uint8")

#Upper wall
lineOfWallParticles(-150, 100, 150, 100, 20)
#Bottom wall
lineOfWallParticles(-150, -100, 150, -100, 20)
#Left wall
lineOfWallParticles(-150, 100, -150, -100, 20)
#Right wall
lineOfWallParticles(150, -100, 150, -30, 10)
lineOfWallParticles(150, 100, 150, 30, 10)
#Exit atractor
exitAtractor = np.array([300.0, 0.0])

for i in range(0, nParticles):
    x = random.randrange(-150 + 2 * wallParticlesSize, -100 + 2 * wallParticlesSize)
    y = random.randrange(-100 + 2 * wallParticlesSize, 100 - 2 * wallParticlesSize)
    speedX = random.random()
    speedY = random.random()
    newParticle = Particle(x, y, speedX, speedY, particleSize)
    particles.append(newParticle)


MaxIterations = 500
NumParticles = len(particles)

for count in range(MaxIterations):
    img[:] = (0, 0, 0)
    for i in range(NumParticles):
        for j in range(NumParticles):
            if i != j:
                Fij = particles[i].calculateForce(particles[j])
                particles[i].force += Fij
                #As we already known we must guide all the particles to the right
                result = exitAtractor - particles[i].position
                magnitudeResult = np.linalg.norm(result)
                if magnitudeResult <= 0:
                    magnitudeResult = 1
                #Calculating the force of the vector
                resultForce = result / magnitudeResult
                #DECAY
                resultForce[0] *= 0.001
                resultForce[1] *= 0.009
                particles[i].force += resultForce

    for particle in particles:
        particle.updateSpeed()
        particle.updatePosition()
        particle.draw(x0, y0, img)
        particle.force[:] = 0

    cv2.putText(img, "Iteration: " + str(count + 1), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    pilIm = PIL.Image.fromarray(img, mode="RGB")
    with BytesIO() as fOut:
        pilIm.save(fOut, format="png")
        byPng = fOut.getvalue()

    # set the png bytes as the image value;
    # this updates the image in the browser.
    wIm.value = byPng

Image(value=b'')

In [ ]:
import cv2
import numpy as np
from IPython import display as display
import ipywidgets as ipw
import PIL
from io import BytesIO
import math
import random
import time 

class Particle:
    MaxV = np.sqrt(2)

    def __init__(self, x, y, speedX, speedY, radius, WallParticle=False):
        self.position = np.array([float(x), float(y)])
        self.speed = np.array([float(speedX), float(speedY)])
        self.WallParticle = WallParticle
        self.force = np.array([0.0, 0.0])
        self.radius = radius

    def normalizeVector(self, x):
        norm = np.linalg.norm(x)
        if norm == 0:
            return x * np.inf
        return x / norm

    def calculateForce(self, r2):
        if self.WallParticle == True:
            return np.array([0.0, 0.0])
        result = self.position - r2.position
        r12magnitude = np.linalg.norm(result)
        if r12magnitude <= r2.radius + self.radius:
            #We get the unitary vector and then we split by the magnitude of R where R is a new vector from r1 to r2
            return self.normalizeVector(result) / (r12magnitude ** 2) * wallParticlesSize * 20
        return np.array([0.0, 0.0])

    def updatePosition(self):
        if self.WallParticle == True:
            return
        self.position += self.speed
        return

    def updateSpeed(self):
        if self.WallParticle == True:
            return
        self.speed += self.force

        vmag = np.linalg.norm(self.speed)

        if vmag > self.MaxV:
            self.speed = self.normalizeVector(self.speed) * self.MaxV
        return

    def draw(self, x0, y0, img):
        if self.WallParticle == True:
            color = (0, 255, 255)
        else:
            color = (255, 255, 255)

        cv2.circle(img, (int(x0 + self.position[0]), int(y0 - self.position[1])), self.radius, color, -1)
        return


particles = []
def lineOfWallParticles(x1, y1, x2, y2, N):
    global particles
    x = np.linspace(x1, x2, N)
    y = np.linspace(y1, y2, N)
    for i in range(N):
        particles.append(Particle(x[i], y[i], 0, 0, wallParticlesSize, WallParticle=True))


wIm = ipw.Image()
display.display(wIm)

maxX = 500
maxY = 500
x0 = int(maxX / 2)
y0 = int(maxY / 2)
nParticles = 70
particleSize = 7
wallParticlesSize = 15

img = np.zeros((500, 500, 3), dtype="uint8")

#Upper wall
lineOfWallParticles(-150, 100, 150, 100, 20)
#Bottom wall
lineOfWallParticles(-150, -100, 150, -100, 20)
#Left wall
lineOfWallParticles(-150, 100, -150, -100, 20)
#Right wall
lineOfWallParticles(150, -100, 150, -30, 10)
lineOfWallParticles(150, 100, 150, 30, 10)
#Exit atractor
exitAtractor = np.array([300.0, 0.0])
#Obstacle
particles.append(Particle(100, 0, 0, 0, wallParticlesSize, WallParticle=True))

for i in range(0, nParticles):
    x = random.randrange(-150 + 2 * wallParticlesSize, -100 + 2 * wallParticlesSize)
    y = random.randrange(-100 + 2 * wallParticlesSize, 100 - 2 * wallParticlesSize)
    speedX = random.random()
    speedY = random.random()
    newParticle = Particle(x, y, speedX, speedY, particleSize)
    particles.append(newParticle)


MaxIterations = 500
NumParticles = len(particles)

for count in range(MaxIterations):
    img[:] = (0, 0, 0)
    for i in range(NumParticles):
        for j in range(NumParticles):
            if i != j:
                Fij = particles[i].calculateForce(particles[j])
                particles[i].force += Fij
                #As we already known we must guide all the particles to the right
                result = exitAtractor - particles[i].position
                magnitudeResult = np.linalg.norm(result)
                if magnitudeResult <= 0:
                    magnitudeResult = 1
                #Calculating the force of the vector
                resultForce = result / magnitudeResult
                resultForce[0] *= 0.001
                resultForce[1] *= 0.009
                particles[i].force += resultForce

    for particle in particles:
        particle.updateSpeed()
        particle.updatePosition()
        particle.draw(x0, y0, img)
        particle.force[:] = 0

    cv2.putText(img, "Iteration: " + str(count + 1), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    pilIm = PIL.Image.fromarray(img, mode="RGB")
    with BytesIO() as fOut:
        pilIm.save(fOut, format="png")
        byPng = fOut.getvalue()

    # set the png bytes as the image value;
    # this updates the image in the browser.
    wIm.value = byPng

Image(value=b'')